## Analyze A/B Test Results

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, I will be working to understand the results of an A/B test run by an e-commerce website.  My goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.


<a id='probability'></a>
#### Part I - Probability


In [143]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  
a. Read in the dataset and take a look at the top few rows here:

In [144]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [145]:
df.shape[0]

294478

c. The number of unique users in the dataset.

In [146]:
df.user_id.nunique()

290584

d. The proportion of users converted.

In [147]:
#probabilty of converting = no. of converted / no of data frame indeces 
len(df.query('converted == 1')) / len(df2)

0.12126228126021646

e. The number of times the `new_page` and `treatment` don't line up.

In [148]:
query_treat_old=df.query('group == "treatment" & landing_page == "old_page"')
query_control_new=df.query('group == "control" & landing_page == "new_page"')

In [149]:
treatment_old_nu =len(query_treat_old) 
treatment_new_nu =len(query_control_new)

In [150]:
Missalinment_nu = treatment_old_nu+treatment_new_nu
Missalinment_nu

3893

f. Do any of the rows have missing values?

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


- there are no missing values as all have same number of indeces = 294478

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [152]:
# df2 from df where we excluded the 3893 records where new_page and trement didn't line up
df2 = df.drop((query_treat_old.index) | (query_control_new.index))
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290585 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [153]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] != 'new_page'))].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [154]:
df2.nunique()

user_id         290584
timestamp       290585
group                2
landing_page         2
converted            2
dtype: int64

b. There is one **user_id** repeated in **df2**.  What is it?

In [155]:
# as we see there's one dubplicate of index 1899 & 2893
df2[df2.duplicated(['user_id'],keep=False)]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


c. What is the row information for the repeat **user_id**? 

In [156]:
df2.shape

(290585, 5)

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [157]:
df2 = df2.drop_duplicates(subset='user_id');

In [158]:
# after deleted row
df2.shape

(290584, 5)

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
# length of converted over len of df2
len(df2.query('converted == 1'))/len(df2)

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
len(df2.query('group == "control" & converted == 1')) / len(df2.query('group == "control"'))

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
#same as above
len(df2.query('group == "treatment" & converted == 1')) / len(df2.query('group == "treatment"'))

d. What is the probability that an individual received the new page?

In [ ]:
len(df2.query('landing_page == "new_page"'))/len(df2)

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

the control group has a slightly higher conversion rate (0.1204) than the treatment group (0.1195), however, these results don't provide a solid evidence if one page leads to more conversions.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

Null hypothesis means the conversion rate of the old_page is greater or the same than the conversion rate of the newpage.        
Alternative hypothesis means the conversion rate of the old_page is less than the conversion rate of the newpage.

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
p_new = df2.converted.mean()
p_new

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2.converted.mean()
p_old

c. What is $n_{new}$?

In [ ]:
n_new =len(df2.query('landing_page == "new_page"'))
n_new

d. What is $n_{old}$?

In [ ]:
n_old= len(df2.query('landing_page == "old_page"'))
n_old

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.binomial(n_new,p_new)
new_page_converted

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.binomial(n_old,p_old)
old_page_converted

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
p_diff = (new_page_converted/n_new)-(old_page_converted/n_old)
p_diff

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [ ]:
p_diffs =[]
for _ in range(10000):
    new_converted_simulation = np.random.binomial(n_new,p_new)/n_new
    old_converted_simulation = np.random.binomial(n_old,p_old)/n_old
    diff = (new_converted_simulation - old_converted_simulation) 
    p_diffs.append(diff)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs,bins=30)
plt.xlabel('p_diffs')
plt.ylabel('Freq.')
plt.title('Simulated Difference bet. new_page & old_page under the Null');

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
#The actucl difference:
org_new_mean = df.query('group =="treatment"').converted.mean()
org_old_mean = df.query('group =="control"').converted.mean()
org_diff = (org_new_mean - org_old_mean)
#Convert p_diffs to array to be able to make inequality 
p_diffs = np.array(p_diffs)
# Calculate the propotion of the p_diffs are greater than the actual difference observed in ab_data.csv
(p_diffs > org_diff).mean()

In [ ]:
#line for the original differance 
plt.hist(p_diffs,bins=30);plt.axvline(org_diff,c='orange')
plt.xlabel('p_diffs')
plt.ylabel('Freq.')
plt.title('Simulated Difference bet. new_page & old_page under the Null');

then 89.57% is the proportion of the p_diffs that are greater than the actual difference observed in ab_data.csv.

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

The value above represents the p-value while Null hypothesis is true, we would fail to reject the Null hypothesis and keep the old page.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm
convert_old = sum(df2.query("group == 'control'").converted)
convert_new = sum(df2.query("group == 'treatment'").converted)
n_old = len(df2.query("group == 'control'"))
n_new = len(df2.query("group == 'treatment'"))

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')

z_score, p_value

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

The z-score and the p_value mean that one doesn't reject the Null. The Null being the converted rate of the old_page is the same or greater than the converted rate of the new_page. The p_value is 0.91 and is higher than 0.05 significance level. That means we can be confident with a 95% that the converted rate of the new_page is lower than the old_page.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

logistic regression as the variable is a binary variable (0,1).

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df.head(3)

In [ ]:
df2[['control','treatment']]= pd.get_dummies(df2['group']) 
df2 = df2.drop('control',axis = 1)
df2 = df2.rename(columns={'treatment':'ab_page'})
df2['intercept'] = 1
df2.head(3)

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
#Logit() function for performing logistic regression.
import statsmodels.api as sm

logit = sm.Logit(df2['converted'],df2[['intercept','ab_page']])

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results = logit.fit()
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

- p-value = 0.190
- difference that here is double side check and in part I is one side check  && also due to adding intercept column here.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

We could consider introducing the timestamp or Maybe Age of user if available to determine in which part of the day the individuals converted the most or if age contributes in converting if age is available. For example, if we find that the evening is the period that users spend most of their time on the internet we might also take it into consideration or maybe we find that young users always convert and get updated and old users keep using old and against any change MAYBE .

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_new.head()

In [ ]:
df_new['country'].value_counts()

In [ ]:
### necessary dummy variables
### the two coulumns will be US & CA, as UK won't be necessary as it doesn't represent max or min.
df_new [['US','CA']]= pd.get_dummies(df_new['country'])[['US','CA']]

In [ ]:
df_new.head()

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

In [ ]:
###Linear Model And Obtain the Results
log_mod = sm.Logit(df_new['converted'], df_new[['CA', 'US', 'intercept', 'ab_page']])
results = log_mod.fit()
results.summary()

The p_value for both CA and US terms is higher than 0.05.

<a id='conclusions'></a>
## Conclusions
- At conlusion we can see that the old page was better than the new page, therefore I fail to refuse the null hypothesis, this is based on the probability figures, A/B testand regression.
- There is no strong evidence that the countries (US, CA and UK) influence the conversion rate. as the P-Value of countries didn't pass 0.5 so not close to 1 at all and we can see that CA really accept the NULL hypothesis.
- So the best to develop is to focus on another new landing page.
- And from the regression above we see that the p-value is higher in US than in Canada, which means that users in the US are more likely to convert, but still not enough evidence to reject the null hypothesis.

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])